In [1]:
import random
import numpy as np
import pandas as pd
import pulp
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, LpMaximize

In [2]:
def generate_random_number(seed,begin,end):
    random.seed(seed)  # Set the random seed
    return random.randint(begin, end)



In [3]:
seed = 614 #0614 -> Danick's id 20880614

In [4]:
def generate_instance(n_orders,n_itemtypes,n_totes,max_item_quantity=3)->dict:

    order_itemtypes = [[] for _ in range(n_orders)]  # Initialize an empty list of lists
    order_quantities = [[] for _ in range(n_orders)]
    for i in range(n_orders):
        order_size = random.randint(1, 3)

        tt = random.sample(range(1, n_itemtypes+1), order_size)
        qq = [random.randint(1, max_item_quantity) for _ in range(order_size)]
        order_itemtypes[i] = (tt)
        order_quantities[i] = (qq)

    orders_totes = [[] for _ in range(n_orders)]
    for i in range(n_orders):
        for j in range(len(order_itemtypes[i])):
            if j == 0:
                orders_totes[i].append(random.randint(1, n_totes))
            else:
                if random.randint(0, 1) == 0:
                    orders_totes[i].append(orders_totes[i][0])
                else:
                    orders_totes[i].append(random.randint(1, n_totes))

    return {
        'orders':[x for x in range(1,1+n_orders)],
        'item_types':[x for x in range(1,1+n_itemtypes)],
        'totes':[x for x in range(1,1+n_totes)],
        'order_item_types':order_itemtypes,
        'order_item_quantities':order_quantities,
        'order_item_totes':orders_totes
    }

In [5]:
small_instance=generate_instance(
        n_orders=6,
        n_itemtypes=5,
        n_totes=8,
        max_item_quantity=2
    )

In [6]:
large_instances=[
    generate_instance(
        n_orders=generate_random_number(seed,10,15),
        n_itemtypes=generate_random_number(seed,7,10),
        n_totes=generate_random_number(seed,15,20)
    )
    for _ 
    in range(10)
]

In [7]:
small_instance

{'orders': [1, 2, 3, 4, 5, 6],
 'item_types': [1, 2, 3, 4, 5],
 'totes': [1, 2, 3, 4, 5, 6, 7, 8],
 'order_item_types': [[3, 1, 2], [3, 5, 4], [5, 2, 4], [4], [2, 1], [3]],
 'order_item_quantities': [[1, 2, 1], [2, 2, 2], [2, 1, 2], [1], [2, 2], [1]],
 'order_item_totes': [[2, 2, 6], [5, 5, 5], [6, 7, 3], [1], [7, 7], [4]]}

In [8]:
class Model:
    def __init__(
        self,
        orders,
        totes,
        order_item_totes,
        order_item_quantities,
        order_item_types,
        item_types,
        use_heuristic=False
    ):
        self.K = orders
        self.J = totes
        self.J0 = [0]+totes
        self.Jn1 = totes+[len(totes)+1]
        self.Jk = order_item_totes

        self.order_item_types = order_item_types
        self.order_item_quantities = order_item_quantities
        self.item_types = item_types

        self.tote_content = [[] for _ in range(len(totes))]
        for order_i, _ in enumerate(order_item_totes):
            for (
                order_item_i, order_item_quantity
            ) in enumerate(order_item_quantities[order_i]):
                self.tote_content[
                    order_item_totes[order_i][order_item_i] -1
                ] += [
                    order_item_types[order_i][order_item_i]
                ]*order_item_quantity

        #self.p = [max(0.01,len(tote)) for tote in self.tote_content]
        self.p = [len(tote) for tote in self.tote_content]

        self.M = sum(len(tote) for tote in self.tote_content) + 1

        self._initiate_model(use_heuristic)

    def _initiate_model(self, use_heuristic=False):

        self.model = LpProblem(sense=LpMinimize)

        # xij
        self.x = [
            [
                LpVariable(f'x_i{i}_j{j}', lowBound=0, cat='Binary')
                for j in range(len(self.J)+2)
            ]
            for i in range(len(self.J)+2)
        ]
        
        # cj
        self.c = [LpVariable(f'c_j{j}', lowBound=0) for j in self.J]
        
        # cok
        self.co = [LpVariable(f'co_j{k}', lowBound=0) for k in self.K]
        
        # objective
        self.model += lpSum(self.co[k_i] for k_i, k in enumerate(self.K))
        
        # constraint 1 -> all totes must be used
        for j in self.J:
            self.model += lpSum(self.x[i][j] for i in self.J0) == 1
        
        # constraint 2 -> one induction line
        self.model += lpSum(self.x[0][j] for j in self.J) == 1
        
        # constraint 3 -> all preceded tote must precede
        for j in self.J:
            self.model += lpSum(
                self.x[i][j] for i in self.J0
            )-lpSum(
                self.x[j][i] for i in self.Jn1
            ) == 0
        
        # constraint 4 -> first tote completion time is it's own
        for j_i, j in enumerate(self.J):
            self.model += self.c[j_i] >= self.p[j_i]*self.x[0][j]
        
        # constraint 5 -> tote completion time is itself plus all preceding
        for j_i, j in enumerate(self.J):
            for i_i, i in enumerate(self.J):
                self.model += self.c[j_i] >= (
                    (self.c[i_i]+self.p[j_i])-self.M*(1-self.x[i][j])
                )
        
        # constraint 6 -> order completion is its latest completed tote
        for k_i, k in enumerate(self.K):
            for tote in self.Jk[k_i]:
                self.model += self.co[k_i] >= self.c[tote-1]

        if use_heuristic:
            order_tote_count=[len(order_totes) for order_totes in self.Jk]
            max_tote_count=max(order_tote_count)

            #add totes of orders with the fewest tote first
            solution=[]
            for tote_count in range(1,max_tote_count+1):
                for order_totes in self.Jk:
                    if len(order_totes)==tote_count:
                        for tote in order_totes:
                            if not tote in solution:
                                solution.append(tote)
            #add empty totes at the end
            for empty_tote in [
                tote for tote in self.J if tote not in solution
            ]:
                solution.append(empty_tote)

            prev=0
            for tote in solution:
                self.model += self.x[prev][tote] == 1 
                prev=tote
            self.model += self.x[prev][self.Jn1[-1]] == 1

    def solve(self, solver='CPLEX_CMD'):
        self.model.solve(solver=pulp.getSolver(solver))

        print('Objective: ' + str(pulp.value(self.model.objective)))

    def reset_model(self):
        self._initiate_model()

    @property
    def objective(self):
        return round(pulp.value(self.model.objective))
    
    def _value(self,x):
            v=pulp.value(x)
            if v:
                return round(v)
            else:
                return 0
    
    def get_x_value(self):
        
            
        x = [
            [
                self._value(self.x[i][j])
                for j in range(len(self.J)+2)
            ]
            for i in range(len(self.J)+2)
        ]

        return x
    
    def get_co_value(self):
        return [self._value(self.co[k_i]) for k_i,_ in enumerate(self.K)]
    
    def get_c_value(self):
        return [self._value(self.c[j_i]) for j_i,_ in enumerate(self.J)]

    @property
    def solution(self):
        """
        Returns the order of the totes first one on the left.
        """
        x = self.get_x_value()

        totes = []

        next = 0
        while next != self.Jn1[-1]:
            index = x[next].index(1)
            totes.append(index)
            next = index

        return totes[:-1]

    def get_simulation_input(self, conveyor_count=4):
        # conv_num,cirle,pentagon,trapezoid,triangle,star,moon,heart,cross
        assignment = [[] for _ in range(conveyor_count)]
        for k in self.K:
            assignment[(k-1) % 4].append(k)

        conveyor_picklist = [
            [0 for _ in range(len(self.item_types)+1)] 
            for _ in range(conveyor_count)
        ]
        for conveyor_i, conveyor in enumerate(assignment):
            for order in conveyor:
                order_i = order-1
                for item_i, type in enumerate(self.order_item_types[order_i]):
                    conveyor_picklist[conveyor_i][type] += (
                        self.order_item_quantities[order_i][item_i]
                    )
        for index, picklist in enumerate(conveyor_picklist):
            picklist[0]=index


        if len(self.item_types) == 5:
            columns = ['conv_num', 'cirle', 'cross',
                       'heart', 'star', 'triangle']
        else:
            columns = ['conv_num'] + [
                'shape' + str(type) for type in self.item_types
            ]

        induction_line_items = sum(
            [self.tote_content[tote_i] 
            for tote_i, _ in enumerate(self.solution)
            ],
            []
        )
        induction_line_input = [
            columns[item_type] 
            for item_type in induction_line_items
        ]

        return {
            'input_dataframe': pd.DataFrame(conveyor_picklist, columns=columns),
            'order_conveyor_assignment': assignment,
            'induction_line_input': induction_line_input
        }

In [9]:
small_model=Model(use_heuristic=True,**small_instance)

In [10]:
small_model.tote_content

[[4],
 [3, 1, 1],
 [4, 4],
 [3],
 [3, 3, 5, 5, 4, 4],
 [2, 5, 5],
 [2, 2, 2, 1, 1],
 []]

In [11]:
small_model.Jk

[[2, 2, 6], [5, 5, 5], [6, 7, 3], [1], [7, 7], [4]]

In [12]:
small_model.solve(solver='CPLEX_CMD')

Objective: 63.0


In [13]:
small_model.objective

63

In [14]:
small_model.get_x_value()

[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [15]:
small_model.get_c_value()

[1, 10, 21, 2, 19, 13, 7, 21]

In [16]:
small_model.get_co_value()

[13, 19, 21, 1, 7, 2]

In [17]:
small_model.solution

[1, 4, 7, 2, 6, 5, 3, 8]

In [18]:
input=small_model.get_simulation_input()
input.keys()

dict_keys(['input_dataframe', 'order_conveyor_assignment', 'induction_line_input'])

In [19]:
input['input_dataframe']

,conv_num,cirle,cross,heart,star,triangle
0,0,4,3,1,0,0
1,1,0,0,3,2,2
2,2,0,1,0,2,2
3,3,0,0,0,1,0


In [20]:
input['input_dataframe'].to_csv('simulation_input.csv',index=False)

In [21]:
input['order_conveyor_assignment']

[[1, 5], [2, 6], [3], [4]]

In [22]:
input['induction_line_input']

['star',
 'heart',
 'cirle',
 'cirle',
 'star',
 'star',
 'heart',
 'heart',
 'heart',
 'triangle',
 'triangle',
 'star',
 'star',
 'cross',
 'triangle',
 'triangle',
 'cross',
 'cross',
 'cross',
 'cirle',
 'cirle']

In [28]:
temp=large_instances[0]
big_model=Model(use_heuristic=False,**temp)

In [24]:
temp

{'orders': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
 'item_types': [1, 2, 3, 4, 5, 6, 7, 8, 9],
 'totes': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17],
 'order_item_types': [[4],
  [2, 9, 1],
  [3, 7, 4],
  [6, 7],
  [6, 4, 2],
  [9, 8],
  [7, 2, 4],
  [1, 3],
  [9, 3, 2],
  [4],
  [6],
  [8, 1, 9]],
 'order_item_quantities': [[1],
  [3, 1, 1],
  [1, 2, 3],
  [2, 2],
  [1, 3, 1],
  [2, 1],
  [2, 2, 3],
  [1, 2],
  [3, 1, 2],
  [2],
  [1],
  [3, 3, 2]],
 'order_item_totes': [[13],
  [15, 15, 15],
  [11, 3, 11],
  [7, 7],
  [17, 13, 10],
  [7, 14],
  [5, 7, 5],
  [14, 17],
  [5, 15, 5],
  [17],
  [5],
  [4, 4, 4]]}

In [25]:
big_model.Jk

[[13],
 [15, 15, 15],
 [11, 3, 11],
 [7, 7],
 [17, 13, 10],
 [7, 14],
 [5, 7, 5],
 [14, 17],
 [5, 15, 5],
 [17],
 [5],
 [4, 4, 4]]

In [26]:
big_model.tote_content

[[],
 [],
 [7, 7],
 [8, 8, 8, 1, 1, 1, 9, 9],
 [7, 7, 4, 4, 4, 9, 9, 9, 2, 2, 6],
 [],
 [6, 6, 7, 7, 9, 9, 2, 2],
 [],
 [],
 [2],
 [3, 4, 4, 4],
 [],
 [4, 4, 4, 4],
 [8, 1],
 [2, 2, 2, 9, 1, 3],
 [],
 [6, 3, 3, 4, 4]]

In [ ]:
big_model.solve(solver='PULP_CBC_CMD')